# ![nrg](https://www.gravatar.com/avatar/6282094b092c756acc9f7552b164edfe?s=24) NRG Cloud | API Example Notebook

The following is an example Jupyter Notebook for accessing NRG Cloud APIs to 

- View site information
- Upload data files
- Convert raw data files
- Export ranges of measurement and diagnostic data

Login to NRG Cloud and get your `client_id` and `client_secret` from the API Setup page: https://cloud.nrgsystems.com/data-manager/api-setup

*This notebook requires that you have access to sites within the NRG Cloud platform.*

In [ ]:
import nrgpy
import pandas as pd

In [ ]:
client_id = 'your-client-id-here'
client_secret = 'your-client-secret-here'

txt_output_dir = 'exports'

## View site information

In [ ]:
sites = nrgpy.CloudSites(
    client_id=client_id,
    client_secret=client_secret
)

In [ ]:
sites.sites_df

## Upload data files

Upload data files for any site currently in your sites list. 

_Files from sites not configured in NRG Cloud will not be imported._

In [ ]:
path_to_rld_files = "/path/to/rld/files/"

start_date = "2019-04-20"
end_date = "2019-04-30"

importer = nrgpy.CloudImport(
                client_id=client_id,
                client_secret=client_secret,
                in_dir=path_to_rld_files,
                start_date=start_date,
                end_date=end_date,
                file_filter=""
            )

## Export a range of measurement data

In [ ]:
site_id = 175    # get from site_df in example above
output_txt_dir = "txt_files"

start_date = "2020-04-10"
end_date = "2020-05-01"

In [ ]:
exporter = nrgpy.CloudExportJob(
            client_id=client_id,
            client_secret=client_secret,
            out_dir=output_txt_dir,
            site_id=site_id,
            start_date=start_date,
            end_date=end_date,
            file_format="singleFile",     # singleFile for TXT, or multipleFiles for RLD and original ZX files
            export_type="measurements",   # "measurements" or "events" (SymPRO sites support "diagnostic" and "communication" as well)
            unzip=True,
        )

exporter.create_export_job()
exporter.monitor_export_job(download=True)

## Read exported data

Use nrgpy reader function to create dataframe objects

In [ ]:
reader = nrgpy.SymProTextRead(exporter.export_filepath, ch_details=True)

In [ ]:
reader.ch_info

In [ ]:
reader.data.head()

In [ ]:
nrgpy.check_intervals(reader.data)

## Convert raw data files

In [ ]:
in_dir = "/path/to/rld/files"

In [ ]:
converter = nrgpy.CloudConvert(
    client_id=client_id,
    client_secret=client_secret,
    rld_dir=in_dir,
    start_date=start_date,
    end_date=end_date,
    out_dir=output_txt_dir,
)

In [ ]:
converter.process()